In [82]:
# Params
dataPath = "./data/"
resultPath = "./results/tables/"
filenames = ["SQLite","Dune"]

In [69]:
# Functions to manipulate data and dataframes
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np

# Select the files you will use
def getFiles(baseFilename):
    returnFiles = []
    
    files = [f for f in listdir(dataPath) if isfile(join(dataPath, f)) and f.startswith(baseFilename)]
    files.sort()
    for filename in files:
        # Here only appear files starting with the value in baseFilename
        # You can add several filters as you want, example here by excluding file with a certain version number : 
        '''
        # Split the last one
        splitted = filename.split("-")
        # Get the last version
        num = splitted[len(splitted)-1].split(".")[0]
        
        if num == "0001":
            continue
        '''
        #Or simply excluding file by its name
        '''
        if filename == "SQLite-0002.csv":
            continue
        '''
        
        returnFiles.append(dataPath+filename)
        
    return returnFiles


# baseFilename is the root of the fiels considered
# dfResults is the dataframe create from the results-list.csv, used to link the results of a configuration to the params of the configuration
# params is the list of params of the configuration to show (max_features,max_depth...)

# This function creates a dataframe, each row being a configuration and each column a mean or a standard deviation of a metric
def totalMeanDataframe(baseFilename, dfResults, params):
    metrics = ["accuracy","specificity","precision","recall","npv","negNPV","negAccuracy"]
    averages=[]
    averageBase = {}
    for m in metrics:
        averageBase[m] = 0


    for filename in getFiles(baseFilename):
        df = pd.read_csv(filename)

        df['accuracy']= (df['TP']+df['TN']) / (df['TP']+df['TN']+df['FN']+df['FP'])
        df['specificity']=df['TN']/(df['TN']+df['FP'])
        df['precision']=df['TP']/(df['TP'] + df['FP'])
        df['recall']= df['TP'] / (df['TP']+ df['FN']) # Recall of class 1
        df['npv']=df['TN']/(df['TN']+df['FN'])

        df['index']=df.groupby('sr').cumcount()+1
        df['frequency']=max(df['index'])
        df['index']= df['index'] * 100 / df['frequency']

        df['negNPV']=df['npv']-(df['index']/100)
        df['negAccuracy']=df['accuracy']-(df['index']/100)

        average = dict(averageBase)
        
        for m in metrics:
            average[m] = df[m].mean()
            average[m+"-std"] = df[m].std()
        
        for param in params:
            average[param] = dfResults[dfResults["results"] == filename][param].values[0]
            
        averages.append(average)

    return pd.DataFrame(averages)

# Used to sort columns so that params columns are not merged with thresholds ones, the latters being sorted by value.
def sort_columns_with_str(columns):
    strList = []
    intList = []
    for i in columns:
        if type(i) is str:
            strList.append(i)
        elif type(i) is np.int64:
            intList.append(i)
    return strList+sorted(intList)

# This function creates a list of dataframes, each dataframe about a metric, each row being a configuration 
# and each column a mean of the metric considering a threshold (number of system configuration considered as acceptable)
def tableByMetric(baseFilename, dfResults, params):
    dfs = {}
    for m in metrics:
        dfs[m] = []


    for filename in getFiles(baseFilename):
        df = pd.read_csv(filename)

        df['accuracy']= (df['TP']+df['TN']) / (df['TP']+df['TN']+df['FN']+df['FP'])
        df['specificity']=df['TN']/(df['TN']+df['FP'])
        df['precision']=df['TP']/(df['TP'] + df['FP'])
        df['recall']= df['TP'] / (df['TP']+ df['FN']) # Recall of class 1
        df['npv']=df['TN']/(df['TN']+df['FN'])

        df['index']=df.groupby('sr').cumcount()+1
        df['frequency']=max(df['index'])
        df['index']= df['index'] * 100 / df['frequency']

        df['negNPV']=df['npv']-(df['index']/100)
        df['negAccuracy']=df['accuracy']-(df['index']/100)

        for m in metrics:
            df2 = df.groupby("sr")[m].mean()
            temp = {}
            for i,x in enumerate(df2):
                temp[df2.index[i]] = x
                
            for param in params:
                temp[param] = dfResults[dfResults["results"] == filename][param].values[0]
                
            dfs[m].append(temp.copy())
    
    for i in dfs:
        dfs[i] = pd.DataFrame(dfs[i])
        dfs[i] = dfs[i][sort_columns_with_str(dfs[i].columns)]
    
    return dfs

In [74]:
#Function to apply colors and dataframes
from matplotlib import colors
import matplotlib.pyplot as plt

# Here are the functions to color the dataframes tables

# Function to color cells like a heatmap for th whole table
def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

# Functions for a colorscale that apply column by column
def getColor(val,min,max):
    
    if np.isnan(val):
        return "white"
    
    aR = 0
    aG = 255
    aB = 0
    bR = 255
    bG = 0
    bB = 0
    
    value = (val-min)/(max-min)
    
    red = hex(int((bR - aR) * value + aR))
    if red == "0x0":
        red="00"
    elif len(red) == 3:
        red = "0"+red.lstrip("0x")
    else:
        red = red.lstrip("0x")
    green = hex(int((bG - aG) * value + aG))
    if green == "0x0":
        green="00"
    elif len(green) == 3:
        green = "0"+green.lstrip("0x")
    else:
        green = green.lstrip("0x")
    blue = hex(int((bB - aB) * value + aB))
    if blue == "0x0":
        blue="00"
    elif len(blue) == 3:
        blue = "0"+blue.lstrip("0x")
    else:
        blue = blue.lstrip("0x")
    
    return "#" + red + green + blue

def highlight_colorscale(s):
    colors = [getColor(i,s.min(),s.max()) for i in s]
    #print(colors)
    return ['background-color: '+c for c in colors]

def highlight_colorscale_reversed(s):
    colors = [getColor(i,s.max(),s.min()) for i in s]
    #print(colors)
    return ['background-color: '+c for c in colors]


In [56]:
# Functions to save styled dataframes in files
def saveStyleToHtml(style, filename):
    html = style.render()
    with open(filename,'w') as f:
        f.write(html)
        f.close()
        pass

In [50]:
metrics = ["accuracy","specificity","precision","recall","npv","negNPV","negAccuracy"]
metricsStd = ["accuracy-std","specificity-std","precision-std","recall-std","npv-std","negNPV-std","negAccuracy-std"]

dfResults = pd.read_csv(dataPath+"results-list.csv")
params=["max_features","min_samples_leaf"]

In [51]:
df = totalMeanDataframe("SQLite",dfResults,params)
# To highlight min and max values
# subset is used to specify which column are concerned by coloring
df.style.highlight_min(subset=metricsStd).highlight_max(subset=metrics)

,accuracy,accuracy-std,max_features,min_samples_leaf,negAccuracy,negAccuracy-std,negNPV,negNPV-std,npv,npv-std,precision,precision-std,recall,recall-std,specificity,specificity-std
0,0.839985,0.085526,nan,2,0.327164,0.283094,0.216816,0.147674,0.73012,0.196725,0.804535,0.123079,0.733882,0.166085,0.773802,0.190427
1,0.872158,0.0988315,0.8,1,0.359337,0.295043,0.284897,0.212696,0.798241,0.189859,0.832127,0.147565,0.817192,0.14911,0.802206,0.191363
2,0.839078,0.0854003,0.8,2,0.326258,0.283089,0.215512,0.146662,0.728545,0.196937,0.802877,0.123289,0.732824,0.165787,0.770553,0.19312
3,0.872307,0.0990815,0.9,1,0.359487,0.295149,0.284944,0.212535,0.798385,0.189839,0.832607,0.146748,0.817765,0.148214,0.802331,0.191914
4,0.839631,0.0853874,0.9,2,0.32681,0.282974,0.216331,0.147197,0.729487,0.196864,0.803759,0.123597,0.73369,0.165076,0.772606,0.191035
5,0.872216,0.0994757,nan,1,0.359396,0.295154,0.284631,0.21264,0.7978,0.191055,0.832607,0.147213,0.817784,0.147843,0.802952,0.191138


In [70]:
# To create a colorscale by value
sty = df.style.apply(highlight_colorscale, subset=metrics).apply(highlight_colorscale_reversed, subset=metricsStd)
saveStyleToHtml(sty, "test.html")
sty

,accuracy,accuracy-std,max_features,min_samples_leaf,negAccuracy,negAccuracy-std,negNPV,negNPV-std,npv,npv-std,precision,precision-std,recall,recall-std,specificity,specificity-std
0,0.839985,0.085526,nan,2,0.327164,0.283094,0.216816,0.147674,0.73012,0.196725,0.804535,0.123079,0.733882,0.166085,0.773802,0.190427
1,0.872158,0.0988315,0.8,1,0.359337,0.295043,0.284897,0.212696,0.798241,0.189859,0.832127,0.147565,0.817192,0.14911,0.802206,0.191363
2,0.839078,0.0854003,0.8,2,0.326258,0.283089,0.215512,0.146662,0.728545,0.196937,0.802877,0.123289,0.732824,0.165787,0.770553,0.19312
3,0.872307,0.0990815,0.9,1,0.359487,0.295149,0.284944,0.212535,0.798385,0.189839,0.832607,0.146748,0.817765,0.148214,0.802331,0.191914
4,0.839631,0.0853874,0.9,2,0.32681,0.282974,0.216331,0.147197,0.729487,0.196864,0.803759,0.123597,0.73369,0.165076,0.772606,0.191035
5,0.872216,0.0994757,nan,1,0.359396,0.295154,0.284631,0.21264,0.7978,0.191055,0.832607,0.147213,0.817784,0.147843,0.802952,0.191138


In [76]:
dfs = tableByMetric("SQLite",dfResults,params)
df = dfs["accuracy"]
df.style

,min_samples_leaf,max_features,1,46,91,136,181,226,271,316,361,406,451,496,541,586,631,676,721,766,811,856,901,946,991,1036,1081,1126,1171,1216,1261,1306,1351,1396,1441,1486,1531,1576,1621,1666,1711,1756,1801,1846,1891,1936,1981,2026,2071,2116,2161,2206,2251,2296,2341,2386,2431,2476,2521,2566,2611,2656,2701,2746,2791,2836,2881,2926,2971,3016,3061,3106,3151,3196,3241,3286,3331,3376,3421,3466,3511,3556,3601,3646,3691,3736,3781,3826,3871,3916,3961,4006,4051,4096,4141,4186,4231,4276,4321,4366,4411,4456,4501,4546
0,2,nan,0.664233,0.68858,0.712358,0.72274,0.733548,0.742949,0.750191,0.755866,0.759566,0.764657,0.770771,0.77334,0.777044,0.780665,0.78273,0.785964,0.790622,0.792379,0.79676,0.797767,0.801377,0.804361,0.806108,0.808712,0.810223,0.813145,0.814276,0.817643,0.818873,0.821163,0.823881,0.826048,0.82689,0.829031,0.830186,0.831691,0.834087,0.835757,0.836904,0.838449,0.840003,0.841028,0.84248,0.844297,0.845474,0.846653,0.848178,0.849767,0.85065,0.851198,0.853251,0.853831,0.855156,0.856424,0.857792,0.858576,0.859611,0.860909,0.861807,0.862939,0.863373,0.864746,0.866039,0.867142,0.86776,0.868646,0.869905,0.870853,0.871901,0.87222,0.873514,0.874323,0.875205,0.875845,0.876832,0.87777,0.878629,0.879443,0.880143,0.880848,0.882179,0.882885,0.883655,0.884574,0.885491,0.886641,0.887179,0.887801,0.888767,0.889722,0.890645,0.891815,0.892638,0.893294,0.894288,0.895397,0.896381,0.89746,0.898501,0.899703,0.900699,0.902033
1,1,0.8,0.656407,0.691055,0.712396,0.729445,0.739243,0.746654,0.752828,0.758397,0.764939,0.767661,0.772094,0.776904,0.780546,0.783513,0.786093,0.789701,0.792261,0.796665,0.798593,0.801325,0.803866,0.807258,0.809171,0.811915,0.814909,0.817636,0.820114,0.822473,0.824779,0.827695,0.829935,0.832202,0.834424,0.836138,0.838925,0.841066,0.843817,0.845563,0.848171,0.850215,0.852878,0.854477,0.856929,0.859068,0.861317,0.86353,0.86595,0.868082,0.87071,0.872024,0.874499,0.876812,0.878905,0.881383,0.883682,0.88542,0.887448,0.889302,0.892058,0.894285,0.895826,0.899077,0.901232,0.903141,0.905491,0.907891,0.910185,0.91271,0.914741,0.917428,0.919495,0.921646,0.924261,0.926836,0.928979,0.931567,0.933957,0.93647,0.938682,0.941204,0.94345,0.946447,0.948933,0.950969,0.953762,0.956363,0.959167,0.961576,0.964074,0.96673,0.969507,0.972049,0.974774,0.977472,0.980123,0.982821,0.985609,0.988359,0.991152,0.993945,0.996691,0.999547
2,2,0.8,0.646285,0.690993,0.712269,0.726553,0.737702,0.745916,0.752112,0.758543,0.761503,0.764744,0.770584,0.773094,0.778778,0.782854,0.784141,0.788122,0.790237,0.793017,0.796063,0.798472,0.800818,0.803337,0.806233,0.807951,0.811295,0.812719,0.814997,0.817004,0.818385,0.820918,0.822398,0.825641,0.826612,0.827837,0.829632,0.83178,0.833475,0.83454,0.836389,0.837955,0.839233,0.841196,0.841517,0.843372,0.844862,0.846283,0.847453,0.849011,0.850486,0.850817,0.85222,0.853466,0.854898,0.855742,0.856636,0.857596,0.858436,0.859853,0.860883,0.862097,0.862598,0.86371,0.864572,0.865876,0.866463,0.867257,0.86884,0.869594,0.870299,0.871202,0.872023,0.872922,0.873543,0.874643,0.875379,0.876137,0.876941,0.877987,0.87879,0.879311,0.880332,0.881008,0.881918,0.88268,0.883644,0.884666,0.885461,0.886127,0.886722,0.88785,0.88867,0.889332,0.890081,0.890899,0.891743,0.892754,0.893641,0.894689,0.895541,0.896243,0.897573,0.898345
3,1,0.9,0.657426,0.690872,0.712112,0.727274,0.739693,0.746621,0.753754,0.759544,0.763246,0.769604,0.77288,0.775923,0.780312,0.784103,0.786355,0.790106,0.792538,0.795915,0.798825,0.800818,0.803887,0.806286,0.809848,0.812328,0.81457,0.817762,0.819772,0.822621,0.824484,0.827668,0.829847,0.832605,0.834217,0.836931,0.839208,0.840941,0.843261,0.845747,0.84901,0.850028,0.852736,0.854241,0.856986,0.859128,0.860966,0.863669,0.865495,0.868172,0.870697,0.872974,0.874345,0.876896,0.879723,0.88141,0.884067,0.885706,0.887886,0.890634,0.892265,0.894749,0.89711,0.899105,0.901139,0.903661,0.90581,0.908201,0.910283,0.912466,0.915441,0.917527,0.919416,0.922147,0.924604,0.926649,0.929502,0.93169,0.93406,0.936593,0.939051,0.941339,0.944134,0.946339

In [77]:
# Colors params
df.style.apply(highlight_colorscale, subset=params)

,min_samples_leaf,max_features,1,46,91,136,181,226,271,316,361,406,451,496,541,586,631,676,721,766,811,856,901,946,991,1036,1081,1126,1171,1216,1261,1306,1351,1396,1441,1486,1531,1576,1621,1666,1711,1756,1801,1846,1891,1936,1981,2026,2071,2116,2161,2206,2251,2296,2341,2386,2431,2476,2521,2566,2611,2656,2701,2746,2791,2836,2881,2926,2971,3016,3061,3106,3151,3196,3241,3286,3331,3376,3421,3466,3511,3556,3601,3646,3691,3736,3781,3826,3871,3916,3961,4006,4051,4096,4141,4186,4231,4276,4321,4366,4411,4456,4501,4546
0,2,nan,0.664233,0.68858,0.712358,0.72274,0.733548,0.742949,0.750191,0.755866,0.759566,0.764657,0.770771,0.77334,0.777044,0.780665,0.78273,0.785964,0.790622,0.792379,0.79676,0.797767,0.801377,0.804361,0.806108,0.808712,0.810223,0.813145,0.814276,0.817643,0.818873,0.821163,0.823881,0.826048,0.82689,0.829031,0.830186,0.831691,0.834087,0.835757,0.836904,0.838449,0.840003,0.841028,0.84248,0.844297,0.845474,0.846653,0.848178,0.849767,0.85065,0.851198,0.853251,0.853831,0.855156,0.856424,0.857792,0.858576,0.859611,0.860909,0.861807,0.862939,0.863373,0.864746,0.866039,0.867142,0.86776,0.868646,0.869905,0.870853,0.871901,0.87222,0.873514,0.874323,0.875205,0.875845,0.876832,0.87777,0.878629,0.879443,0.880143,0.880848,0.882179,0.882885,0.883655,0.884574,0.885491,0.886641,0.887179,0.887801,0.888767,0.889722,0.890645,0.891815,0.892638,0.893294,0.894288,0.895397,0.896381,0.89746,0.898501,0.899703,0.900699,0.902033
1,1,0.8,0.656407,0.691055,0.712396,0.729445,0.739243,0.746654,0.752828,0.758397,0.764939,0.767661,0.772094,0.776904,0.780546,0.783513,0.786093,0.789701,0.792261,0.796665,0.798593,0.801325,0.803866,0.807258,0.809171,0.811915,0.814909,0.817636,0.820114,0.822473,0.824779,0.827695,0.829935,0.832202,0.834424,0.836138,0.838925,0.841066,0.843817,0.845563,0.848171,0.850215,0.852878,0.854477,0.856929,0.859068,0.861317,0.86353,0.86595,0.868082,0.87071,0.872024,0.874499,0.876812,0.878905,0.881383,0.883682,0.88542,0.887448,0.889302,0.892058,0.894285,0.895826,0.899077,0.901232,0.903141,0.905491,0.907891,0.910185,0.91271,0.914741,0.917428,0.919495,0.921646,0.924261,0.926836,0.928979,0.931567,0.933957,0.93647,0.938682,0.941204,0.94345,0.946447,0.948933,0.950969,0.953762,0.956363,0.959167,0.961576,0.964074,0.96673,0.969507,0.972049,0.974774,0.977472,0.980123,0.982821,0.985609,0.988359,0.991152,0.993945,0.996691,0.999547
2,2,0.8,0.646285,0.690993,0.712269,0.726553,0.737702,0.745916,0.752112,0.758543,0.761503,0.764744,0.770584,0.773094,0.778778,0.782854,0.784141,0.788122,0.790237,0.793017,0.796063,0.798472,0.800818,0.803337,0.806233,0.807951,0.811295,0.812719,0.814997,0.817004,0.818385,0.820918,0.822398,0.825641,0.826612,0.827837,0.829632,0.83178,0.833475,0.83454,0.836389,0.837955,0.839233,0.841196,0.841517,0.843372,0.844862,0.846283,0.847453,0.849011,0.850486,0.850817,0.85222,0.853466,0.854898,0.855742,0.856636,0.857596,0.858436,0.859853,0.860883,0.862097,0.862598,0.86371,0.864572,0.865876,0.866463,0.867257,0.86884,0.869594,0.870299,0.871202,0.872023,0.872922,0.873543,0.874643,0.875379,0.876137,0.876941,0.877987,0.87879,0.879311,0.880332,0.881008,0.881918,0.88268,0.883644,0.884666,0.885461,0.886127,0.886722,0.88785,0.88867,0.889332,0.890081,0.890899,0.891743,0.892754,0.893641,0.894689,0.895541,0.896243,0.897573,0.898345
3,1,0.9,0.657426,0.690872,0.712112,0.727274,0.739693,0.746621,0.753754,0.759544,0.763246,0.769604,0.77288,0.775923,0.780312,0.784103,0.786355,0.790106,0.792538,0.795915,0.798825,0.800818,0.803887,0.806286,0.809848,0.812328,0.81457,0.817762,0.819772,0.822621,0.824484,0.827668,0.829847,0.832605,0.834217,0.836931,0.839208,0.840941,0.843261,0.845747,0.84901,0.850028,0.852736,0.854241,0.856986,0.859128,0.860966,0.863669,0.865495,0.868172,0.870697,0.872974,0.874345,0.876896,0.879723,0.88141,0.884067,0.885706,0.887886,0.890634,0.892265,0.894749,0.89711,0.899105,0.901139,0.903661,0.90581,0.908201,0.910283,0.912466,0.915441,0.917527,0.919416,0.922147,0.924604,0.926649,0.929502,0.93169,0.93406,0.936593,0.939051,0.941339,0.944134,0.946339

In [79]:
# Colors values as a heatmap
ra = [i for i in df.columns if type(i) is np.int64]
df.style.apply(background_gradient,
           cmap='autumn_r',
           m=df[ra].min().min(),
           M=df[ra].max().max(),subset=ra)

,min_samples_leaf,max_features,1,46,91,136,181,226,271,316,361,406,451,496,541,586,631,676,721,766,811,856,901,946,991,1036,1081,1126,1171,1216,1261,1306,1351,1396,1441,1486,1531,1576,1621,1666,1711,1756,1801,1846,1891,1936,1981,2026,2071,2116,2161,2206,2251,2296,2341,2386,2431,2476,2521,2566,2611,2656,2701,2746,2791,2836,2881,2926,2971,3016,3061,3106,3151,3196,3241,3286,3331,3376,3421,3466,3511,3556,3601,3646,3691,3736,3781,3826,3871,3916,3961,4006,4051,4096,4141,4186,4231,4276,4321,4366,4411,4456,4501,4546
0,2,nan,0.664233,0.68858,0.712358,0.72274,0.733548,0.742949,0.750191,0.755866,0.759566,0.764657,0.770771,0.77334,0.777044,0.780665,0.78273,0.785964,0.790622,0.792379,0.79676,0.797767,0.801377,0.804361,0.806108,0.808712,0.810223,0.813145,0.814276,0.817643,0.818873,0.821163,0.823881,0.826048,0.82689,0.829031,0.830186,0.831691,0.834087,0.835757,0.836904,0.838449,0.840003,0.841028,0.84248,0.844297,0.845474,0.846653,0.848178,0.849767,0.85065,0.851198,0.853251,0.853831,0.855156,0.856424,0.857792,0.858576,0.859611,0.860909,0.861807,0.862939,0.863373,0.864746,0.866039,0.867142,0.86776,0.868646,0.869905,0.870853,0.871901,0.87222,0.873514,0.874323,0.875205,0.875845,0.876832,0.87777,0.878629,0.879443,0.880143,0.880848,0.882179,0.882885,0.883655,0.884574,0.885491,0.886641,0.887179,0.887801,0.888767,0.889722,0.890645,0.891815,0.892638,0.893294,0.894288,0.895397,0.896381,0.89746,0.898501,0.899703,0.900699,0.902033
1,1,0.8,0.656407,0.691055,0.712396,0.729445,0.739243,0.746654,0.752828,0.758397,0.764939,0.767661,0.772094,0.776904,0.780546,0.783513,0.786093,0.789701,0.792261,0.796665,0.798593,0.801325,0.803866,0.807258,0.809171,0.811915,0.814909,0.817636,0.820114,0.822473,0.824779,0.827695,0.829935,0.832202,0.834424,0.836138,0.838925,0.841066,0.843817,0.845563,0.848171,0.850215,0.852878,0.854477,0.856929,0.859068,0.861317,0.86353,0.86595,0.868082,0.87071,0.872024,0.874499,0.876812,0.878905,0.881383,0.883682,0.88542,0.887448,0.889302,0.892058,0.894285,0.895826,0.899077,0.901232,0.903141,0.905491,0.907891,0.910185,0.91271,0.914741,0.917428,0.919495,0.921646,0.924261,0.926836,0.928979,0.931567,0.933957,0.93647,0.938682,0.941204,0.94345,0.946447,0.948933,0.950969,0.953762,0.956363,0.959167,0.961576,0.964074,0.96673,0.969507,0.972049,0.974774,0.977472,0.980123,0.982821,0.985609,0.988359,0.991152,0.993945,0.996691,0.999547
2,2,0.8,0.646285,0.690993,0.712269,0.726553,0.737702,0.745916,0.752112,0.758543,0.761503,0.764744,0.770584,0.773094,0.778778,0.782854,0.784141,0.788122,0.790237,0.793017,0.796063,0.798472,0.800818,0.803337,0.806233,0.807951,0.811295,0.812719,0.814997,0.817004,0.818385,0.820918,0.822398,0.825641,0.826612,0.827837,0.829632,0.83178,0.833475,0.83454,0.836389,0.837955,0.839233,0.841196,0.841517,0.843372,0.844862,0.846283,0.847453,0.849011,0.850486,0.850817,0.85222,0.853466,0.854898,0.855742,0.856636,0.857596,0.858436,0.859853,0.860883,0.862097,0.862598,0.86371,0.864572,0.865876,0.866463,0.867257,0.86884,0.869594,0.870299,0.871202,0.872023,0.872922,0.873543,0.874643,0.875379,0.876137,0.876941,0.877987,0.87879,0.879311,0.880332,0.881008,0.881918,0.88268,0.883644,0.884666,0.885461,0.886127,0.886722,0.88785,0.88867,0.889332,0.890081,0.890899,0.891743,0.892754,0.893641,0.894689,0.895541,0.896243,0.897573,0.898345
3,1,0.9,0.657426,0.690872,0.712112,0.727274,0.739693,0.746621,0.753754,0.759544,0.763246,0.769604,0.77288,0.775923,0.780312,0.784103,0.786355,0.790106,0.792538,0.795915,0.798825,0.800818,0.803887,0.806286,0.809848,0.812328,0.81457,0.817762,0.819772,0.822621,0.824484,0.827668,0.829847,0.832605,0.834217,0.836931,0.839208,0.840941,0.843261,0.845747,0.84901,0.850028,0.852736,0.854241,0.856986,0.859128,0.860966,0.863669,0.865495,0.868172,0.870697,0.872974,0.874345,0.876896,0.879723,0.88141,0.884067,0.885706,0.887886,0.890634,0.892265,0.894749,0.89711,0.899105,0.901139,0.903661,0.90581,0.908201,0.910283,0.912466,0.915441,0.917527,0.919416,0.922147,0.924604,0.926649,0.929502,0.93169,0.93406,0.936593,0.939051,0.941339,0.944134,0.946339

In [80]:
# Colors params and values separately, values colored as a heatmap
ra = [i for i in df.columns if type(i) is np.int64]
df.style.apply(highlight_colorscale, subset=params).apply(background_gradient,
           cmap='autumn_r',
           m=df[ra].min().min(),
           M=df[ra].max().max(),
           subset=ra)

,min_samples_leaf,max_features,1,46,91,136,181,226,271,316,361,406,451,496,541,586,631,676,721,766,811,856,901,946,991,1036,1081,1126,1171,1216,1261,1306,1351,1396,1441,1486,1531,1576,1621,1666,1711,1756,1801,1846,1891,1936,1981,2026,2071,2116,2161,2206,2251,2296,2341,2386,2431,2476,2521,2566,2611,2656,2701,2746,2791,2836,2881,2926,2971,3016,3061,3106,3151,3196,3241,3286,3331,3376,3421,3466,3511,3556,3601,3646,3691,3736,3781,3826,3871,3916,3961,4006,4051,4096,4141,4186,4231,4276,4321,4366,4411,4456,4501,4546
0,2,nan,0.664233,0.68858,0.712358,0.72274,0.733548,0.742949,0.750191,0.755866,0.759566,0.764657,0.770771,0.77334,0.777044,0.780665,0.78273,0.785964,0.790622,0.792379,0.79676,0.797767,0.801377,0.804361,0.806108,0.808712,0.810223,0.813145,0.814276,0.817643,0.818873,0.821163,0.823881,0.826048,0.82689,0.829031,0.830186,0.831691,0.834087,0.835757,0.836904,0.838449,0.840003,0.841028,0.84248,0.844297,0.845474,0.846653,0.848178,0.849767,0.85065,0.851198,0.853251,0.853831,0.855156,0.856424,0.857792,0.858576,0.859611,0.860909,0.861807,0.862939,0.863373,0.864746,0.866039,0.867142,0.86776,0.868646,0.869905,0.870853,0.871901,0.87222,0.873514,0.874323,0.875205,0.875845,0.876832,0.87777,0.878629,0.879443,0.880143,0.880848,0.882179,0.882885,0.883655,0.884574,0.885491,0.886641,0.887179,0.887801,0.888767,0.889722,0.890645,0.891815,0.892638,0.893294,0.894288,0.895397,0.896381,0.89746,0.898501,0.899703,0.900699,0.902033
1,1,0.8,0.656407,0.691055,0.712396,0.729445,0.739243,0.746654,0.752828,0.758397,0.764939,0.767661,0.772094,0.776904,0.780546,0.783513,0.786093,0.789701,0.792261,0.796665,0.798593,0.801325,0.803866,0.807258,0.809171,0.811915,0.814909,0.817636,0.820114,0.822473,0.824779,0.827695,0.829935,0.832202,0.834424,0.836138,0.838925,0.841066,0.843817,0.845563,0.848171,0.850215,0.852878,0.854477,0.856929,0.859068,0.861317,0.86353,0.86595,0.868082,0.87071,0.872024,0.874499,0.876812,0.878905,0.881383,0.883682,0.88542,0.887448,0.889302,0.892058,0.894285,0.895826,0.899077,0.901232,0.903141,0.905491,0.907891,0.910185,0.91271,0.914741,0.917428,0.919495,0.921646,0.924261,0.926836,0.928979,0.931567,0.933957,0.93647,0.938682,0.941204,0.94345,0.946447,0.948933,0.950969,0.953762,0.956363,0.959167,0.961576,0.964074,0.96673,0.969507,0.972049,0.974774,0.977472,0.980123,0.982821,0.985609,0.988359,0.991152,0.993945,0.996691,0.999547
2,2,0.8,0.646285,0.690993,0.712269,0.726553,0.737702,0.745916,0.752112,0.758543,0.761503,0.764744,0.770584,0.773094,0.778778,0.782854,0.784141,0.788122,0.790237,0.793017,0.796063,0.798472,0.800818,0.803337,0.806233,0.807951,0.811295,0.812719,0.814997,0.817004,0.818385,0.820918,0.822398,0.825641,0.826612,0.827837,0.829632,0.83178,0.833475,0.83454,0.836389,0.837955,0.839233,0.841196,0.841517,0.843372,0.844862,0.846283,0.847453,0.849011,0.850486,0.850817,0.85222,0.853466,0.854898,0.855742,0.856636,0.857596,0.858436,0.859853,0.860883,0.862097,0.862598,0.86371,0.864572,0.865876,0.866463,0.867257,0.86884,0.869594,0.870299,0.871202,0.872023,0.872922,0.873543,0.874643,0.875379,0.876137,0.876941,0.877987,0.87879,0.879311,0.880332,0.881008,0.881918,0.88268,0.883644,0.884666,0.885461,0.886127,0.886722,0.88785,0.88867,0.889332,0.890081,0.890899,0.891743,0.892754,0.893641,0.894689,0.895541,0.896243,0.897573,0.898345
3,1,0.9,0.657426,0.690872,0.712112,0.727274,0.739693,0.746621,0.753754,0.759544,0.763246,0.769604,0.77288,0.775923,0.780312,0.784103,0.786355,0.790106,0.792538,0.795915,0.798825,0.800818,0.803887,0.806286,0.809848,0.812328,0.81457,0.817762,0.819772,0.822621,0.824484,0.827668,0.829847,0.832605,0.834217,0.836931,0.839208,0.840941,0.843261,0.845747,0.84901,0.850028,0.852736,0.854241,0.856986,0.859128,0.860966,0.863669,0.865495,0.868172,0.870697,0.872974,0.874345,0.876896,0.879723,0.88141,0.884067,0.885706,0.887886,0.890634,0.892265,0.894749,0.89711,0.899105,0.901139,0.903661,0.90581,0.908201,0.910283,0.912466,0.915441,0.917527,0.919416,0.922147,0.924604,0.926649,0.929502,0.93169,0.93406,0.936593,0.939051,0.941339,0.944134,0.946339

In [90]:
import os
# Create files containing configuration heatmap as html table for each metric

for filename in filenames:
    #If data fodler does not exists
    if not os.path.exists(resultPath+filename+"/"):
        try:
            os.makedirs(resultPath+filename+"/")
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise


    dfs = tableByMetric(filename,dfResults,params)
    for metric in dfs:
        ra = [i for i in dfs[metric].columns if type(i) is np.int64]
        s = dfs[metric].style.apply(highlight_colorscale, subset=params).apply(background_gradient,
               cmap='autumn_r',
               m=dfs[metric][ra].min().min(),
               M=dfs[metric][ra].max().max(),
               subset=ra)
        
        saveStyleToHtml(s, resultPath+filename+"/"+metric+".html")

In [89]:
# It is also possible to sort by params, to see a possible trending in results, indicating
df = df.sort_values(["min_samples_leaf"])
df.style.apply(highlight_colorscale, subset=params).apply(background_gradient,
           cmap='autumn_r',
           m=df[ra].min().min(),
           M=df[ra].max().max(),
           subset=ra)

,min_samples_leaf,max_features,1,46,91,136,181,226,271,316,361,406,451,496,541,586,631,676,721,766,811,856,901,946,991,1036,1081,1126,1171,1216,1261,1306,1351,1396,1441,1486,1531,1576,1621,1666,1711,1756,1801,1846,1891,1936,1981,2026,2071,2116,2161,2206,2251,2296,2341,2386,2431,2476,2521,2566,2611,2656,2701,2746,2791,2836,2881,2926,2971,3016,3061,3106,3151,3196,3241,3286,3331,3376,3421,3466,3511,3556,3601,3646,3691,3736,3781,3826,3871,3916,3961,4006,4051,4096,4141,4186,4231,4276,4321,4366,4411,4456,4501,4546
1,1,0.8,0.656407,0.691055,0.712396,0.729445,0.739243,0.746654,0.752828,0.758397,0.764939,0.767661,0.772094,0.776904,0.780546,0.783513,0.786093,0.789701,0.792261,0.796665,0.798593,0.801325,0.803866,0.807258,0.809171,0.811915,0.814909,0.817636,0.820114,0.822473,0.824779,0.827695,0.829935,0.832202,0.834424,0.836138,0.838925,0.841066,0.843817,0.845563,0.848171,0.850215,0.852878,0.854477,0.856929,0.859068,0.861317,0.86353,0.86595,0.868082,0.87071,0.872024,0.874499,0.876812,0.878905,0.881383,0.883682,0.88542,0.887448,0.889302,0.892058,0.894285,0.895826,0.899077,0.901232,0.903141,0.905491,0.907891,0.910185,0.91271,0.914741,0.917428,0.919495,0.921646,0.924261,0.926836,0.928979,0.931567,0.933957,0.93647,0.938682,0.941204,0.94345,0.946447,0.948933,0.950969,0.953762,0.956363,0.959167,0.961576,0.964074,0.96673,0.969507,0.972049,0.974774,0.977472,0.980123,0.982821,0.985609,0.988359,0.991152,0.993945,0.996691,0.999547
3,1,0.9,0.657426,0.690872,0.712112,0.727274,0.739693,0.746621,0.753754,0.759544,0.763246,0.769604,0.77288,0.775923,0.780312,0.784103,0.786355,0.790106,0.792538,0.795915,0.798825,0.800818,0.803887,0.806286,0.809848,0.812328,0.81457,0.817762,0.819772,0.822621,0.824484,0.827668,0.829847,0.832605,0.834217,0.836931,0.839208,0.840941,0.843261,0.845747,0.84901,0.850028,0.852736,0.854241,0.856986,0.859128,0.860966,0.863669,0.865495,0.868172,0.870697,0.872974,0.874345,0.876896,0.879723,0.88141,0.884067,0.885706,0.887886,0.890634,0.892265,0.894749,0.89711,0.899105,0.901139,0.903661,0.90581,0.908201,0.910283,0.912466,0.915441,0.917527,0.919416,0.922147,0.924604,0.926649,0.929502,0.93169,0.93406,0.936593,0.939051,0.941339,0.944134,0.946339,0.948977,0.951778,0.95408,0.95692,0.959094,0.961776,0.964347,0.967037,0.9697,0.97224,0.97468,0.977559,0.980146,0.983138,0.985857,0.98851,0.991261,0.993981,0.996762,0.999561
5,1,nan,0.647054,0.683774,0.712451,0.727247,0.737987,0.746742,0.752446,0.758497,0.762481,0.767685,0.772916,0.774732,0.780045,0.783382,0.785494,0.789864,0.793691,0.794966,0.79773,0.801628,0.803458,0.806808,0.809492,0.812621,0.815284,0.818164,0.819613,0.822579,0.824555,0.82732,0.829577,0.832035,0.834897,0.836113,0.839759,0.841759,0.843313,0.845818,0.84819,0.850095,0.853094,0.854653,0.857678,0.859306,0.861617,0.864092,0.865771,0.868233,0.870641,0.872917,0.874456,0.877284,0.879586,0.881681,0.884368,0.886192,0.887704,0.890724,0.892681,0.894862,0.897302,0.899268,0.901423,0.904077,0.90603,0.908296,0.910863,0.912939,0.915608,0.917911,0.920475,0.922185,0.924717,0.927089,0.929901,0.932085,0.934606,0.936904,0.939489,0.941993,0.944186,0.947013,0.949249,0.952042,0.954458,0.956902,0.959673,0.962087,0.964707,0.967344,0.970005,0.972558,0.974952,0.977797,0.980501,0.98321,0.985836,0.988669,0.991425,0.994111,0.996833,0.999552
0,2,nan,0.664233,0.68858,0.712358,0.72274,0.733548,0.742949,0.750191,0.755866,0.759566,0.764657,0.770771,0.77334,0.777044,0.780665,0.78273,0.785964,0.790622,0.792379,0.79676,0.797767,0.801377,0.804361,0.806108,0.808712,0.810223,0.813145,0.814276,0.817643,0.818873,0.821163,0.823881,0.826048,0.82689,0.829031,0.830186,0.831691,0.834087,0.835757,0.836904,0.838449,0.840003,0.841028,0.84248,0.844297,0.845474,0.846653,0.848178,0.849767,0.85065,0.851198,0.853251,0.853831,0.855156,0.856424,0.857792,0.858576,0.859611,0.860909,0.861807,0.862939,0.863373,0.864746,0.866039,0.867142,0.86776,0.868646,0.869905,0.870853,0.871901,0.87222,0.873514,0.874323,0.875205,0.875845,0.876832,0.87777,0.878629,0.879443,0.880143,0.880848,0.882179,0.882885,0

In [91]:
import os
# Create files containing configuration heatmap as html table for each metric, sorted by params

for filename in filenames:
    #If data fodler does not exists
    if not os.path.exists(resultPath+filename+"/"):
        try:
            os.makedirs(resultPath+filename+"/")
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise


    dfs = tableByMetric(filename,dfResults,params)
    for metric in dfs:
        for param in params:
            # Sort by param
            df = dfs[metric].sort_values(param)
            ra = [i for i in df.columns if type(i) is np.int64]
            s = df.style.apply(highlight_colorscale, subset=params).apply(background_gradient,
                   cmap='autumn_r',
                   m=df[ra].min().min(),
                   M=df[ra].max().max(),
                   subset=ra)

            saveStyleToHtml(s, resultPath+filename+"/"+metric+"-"+param+".html")